In [3]:
import torch
import seqm
from seqm.seqm_functions.constants import Constants
from seqm.Molecule import Molecule
from seqm.ElectronicStructure import Electronic_Structure
seqm.seqm_functions.scf_loop.debug=True # print SCF steps 
seqm.seqm_functions.scf_loop.MAX_ITER=500 # MAX number of SCF iterations

DTYPE = torch.float64
torch.set_default_dtype(DTYPE)
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [4]:
species = torch.as_tensor([[16,16],[22,22],[22,16],[35,17],[24,22]],dtype=torch.int64, device=device)

coordinates = torch.tensor([
        [[0.0000,    0.0,    0.0000],
         [0.0000,    1.2,    0.0000]],
    
        [[0.0000,    0.0,    0.0000],
         [0.0000,    1.2,    0.0000]],
    
        [[0.0000,    0.0,    0.0000],
         [0.0000,    1.2,    0.0000]],
    
        [[0.0000,    0.0,    0.0000],
         [0.0000,    1.2,    0.0000]],
    
        [[0.0000,    0.0,    0.0000],
         [0.0000,    1.2,    0.0000]],
                 ], device=device)


const = Constants().to(device)

seqm_parameters = {
                   'method' : 'PM6',  # AM1, MNDO, PM3, PM6, PM6_SP. PM6_SP is PM6 without d-orbitals. Effectively, PM6 for the first two rows of periodic table
                   'scf_eps' : 1.0e-5,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [0, 0.2], # converger used for scf loop
                                         # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                         # [1], adaptive mixing
                                         # [2], adaptive mixing, then pulay
                   'sp2' : [False, 1.0e-5],  # whether to use sp2 algorithm in scf loop,
                                            #[True, eps] or [False], eps for SP2 conve criteria
                   'learned' : [], # learned parameters name list, e.g ['U_ss']
                   #'parameter_file_dir' : '../seqm/params/', # file directory for other required parameters
                   'pair_outer_cutoff' : 1.0e10, # consistent with the unit on coordinates
                   'eig' : True,
                    "Hf_flag": True,
                    'scf_backward' : 0,
                    'UHF' : False, # open shell is currently not supported for PM6
                   }

molecules = Molecule(const, seqm_parameters, coordinates, species).to(device)
esdriver = Electronic_Structure(seqm_parameters).to(device)
esdriver(molecules)

/Users/vishikh/projects/PYSEQM/seqm/seqm_functions/diag_d.py:109: UserWarning: torch.symeig is deprecated in favor of torch.linalg.eigh and will be removed in a future PyTorch release.
The default behavior has changed from using the upper triangular portion of the matrix by default to using the lower triangular portion.
L, _ = torch.symeig(A, upper=upper)
should be replaced with
L = torch.linalg.eigvalsh(A, UPLO='U' if upper else 'L')
and
L, V = torch.symeig(A, eigenvectors=True)
should be replaced with
L, V = torch.linalg.eigh(A, UPLO='U' if upper else 'L') (Triggered internally at /Users/runner/miniforge3/conda-bld/pytorch-recipe_1675740396714/work/aten/src/ATen/native/BatchLinearAlgebra.cpp:2806.)
  e0,v = torch.symeig(x0,eigenvectors=True,upper=True)


scf direct step  :   0 | E[   4]: -358.65836462 | MAX ΔE[   4]: -81.77464670 | MAX ΔDM[   4]:    0.2552329 | MAX ΔDM_ij[   4]:  1.5960220  | N not converged: 5
scf direct step  :   1 | E[   1]: -208.74995365 | MAX ΔE[   1]: -10.25221378 | MAX ΔDM[   4]:    0.2373771 | MAX ΔDM_ij[   4]:  1.3746724  | N not converged: 5
scf direct step  :   2 | E[   4]: -374.51357253 | MAX ΔE[   4]: -23.77403292 | MAX ΔDM[   4]:    0.2111070 | MAX ΔDM_ij[   4]:  1.3256652  | N not converged: 5
scf direct step  :   3 | E[   4]: -361.67538351 | MAX ΔE[   4]:  12.83818902 | MAX ΔDM[   4]:    0.2170586 | MAX ΔDM_ij[   4]:  1.3941663  | N not converged: 5
scf direct step  :   4 | E[   4]: -378.55304744 | MAX ΔE[   4]: -16.87766393 | MAX ΔDM[   4]:    0.2015102 | MAX ΔDM_ij[   4]:  1.2433544  | N not converged: 5
scf direct step  :   5 | E[   4]: -371.40942608 | MAX ΔE[   4]:   7.14362136 | MAX ΔDM[   4]:    0.1931167 | MAX ΔDM_ij[   4]:  1.1463584  | N not converged: 5
scf direct step  :   6 | E[   4]: -383.1

# Total E

In [5]:
molecules.Etot[0] # S2. MOPAC Etot = -333.8287 EV

tensor(-333.8298)

In [6]:
molecules.Etot[1] # Ti2. MOPAC Etot = -120.0651 EV

tensor(-120.0663)

In [8]:
molecules.Etot[2] # TiS. MOPAC Etot = -221.2701 EV

tensor(-221.2703)

In [9]:
molecules.Etot[3] # BrCl. MOPAC Etot = -458.5855 EV

tensor(-458.5839)

In [10]:
molecules.Etot[4] # CrTi. MOPAC Etot = -150.3163 EV  !!! Wrong nuclear rep parameters

tensor(-255.8133)

# Nuclear repulsion

In [11]:
molecules.Enuc[0] # S2. MOPAC Enuc = 290.8693 EV

tensor(290.8621)

In [12]:
molecules.Enuc[1] # Ti2. MOPAC Enuc = 97.3923 EV

tensor(97.3923)

In [13]:
molecules.Enuc[2] # TiS. MOPAC Enuc = 176.1518 EV

tensor(176.1491)

In [14]:
molecules.Enuc[3] # BrCl. MOPAC Enuc = 391.1161 EV

tensor(391.1087)

In [15]:
molecules.Enuc[4] # CrTi. MOPAC Enuc = 251.0941 EV  !!! Wrong nuclear rep parameters

tensor(145.6016)